In [1]:
import shapely.geometry
import descarteslabs as dl 
from pprint import pprint
import descarteslabs as dl
import os
import numpy as np
from osgeo import gdal
from sklearn.ensemble import RandomForestClassifier
import pickle

## Generate trained model

Write helper functions to rasterize vector data via gdal 

In [2]:
def create_mask_from_vector(vector_data_path, cols, rows, geo_transform,
                            projection, target_value=1):
    """Rasterize the given vector (wrapper for gdal.RasterizeLayer)."""
    data_source = gdal.OpenEx(vector_data_path, gdal.OF_VECTOR)
    layer = data_source.GetLayer(0)
    driver = gdal.GetDriverByName('MEM')  # In memory dataset
    target_ds = driver.Create('', cols, rows, 1, gdal.GDT_UInt16)
    target_ds.SetGeoTransform(geo_transform)
    target_ds.SetProjection(projection)
    gdal.RasterizeLayer(target_ds, [1], layer, burn_values=[target_value])
    return target_ds


def vectors_to_raster(file_paths, rows, cols, geo_transform, projection):
    """Rasterize the vectors in the given directory in a single image."""
    labeled_pixels = np.zeros((rows, cols))
    print
    for i, path in enumerate(file_paths):
        label = i+1
        ds = create_mask_from_vector(path, cols, rows, geo_transform,
                                     projection, target_value=label)
        band = ds.GetRasterBand(1)
        labeled_pixels += band.ReadAsArray()
        ds = None
    return labeled_pixels


def write_geotiff(fname, data, geo_transform, projection):
    """Create a GeoTIFF file with the given data."""
    driver = gdal.GetDriverByName('GTiff')
    rows, cols = data.shape
    dataset = driver.Create(fname, cols, rows, 1, gdal.GDT_Byte)
    dataset.SetGeoTransform(geo_transform)
    dataset.SetProjection(projection)
    band = dataset.GetRasterBand(1)
    band.WriteArray(data)
    dataset = None  # Close the file

Train the model with known images and local training data

In [3]:
north_east = 'landsat:LC08:01:RT:TOAR:meta_LC08_L1TP_172062_20170701_20170701_01_RT_v1'
south_east = 'landsat:LC08:01:RT:TOAR:meta_LC08_L1TP_172063_20170701_20170701_01_RT_v1'
north_west = 'landsat:LC08:01:T1:TOAR:meta_LC08_L1TP_173062_20170606_20170616_01_T1_v1'
south_west = 'landsat:LC08:01:T1:TOAR:meta_LC08_L1TP_173063_20170606_20170616_01_T1_v1'
ids = [south_east,south_west, north_east,north_west]

In [4]:
training_arr, training_meta = dl.raster.ndarray(
    ids,
    bands=['red', 'green', 'blue', 'alpha'],
    scales=[[0,4000], [0, 4000], [0, 4000], None],
    data_type='Float32',
    resolution=351.263936238588940,
    srs = "EPSG:32636",
    bounds=(55210.738, -494687.129, 261010.738, -255587.129)
)

In [5]:
rows, cols, n_bands = training_arr.shape 
geo_transform = training_meta['geoTransform']
print(geo_transform)
proj = 'PROJCS["WGS 84 / UTM zone 36N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",33],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32636"]]'

[55210.738, 351.263936239, 0.0, -255587.129, 0.0, -351.263936239]


In [6]:
train_data_path = "data/training/urban/"

In [7]:
files = [f for f in os.listdir(train_data_path) if f.endswith('.shp')]
classes = [f.split('.')[0] for f in files]
print("The class are {} and {}.".format(classes[0],classes[1]))
shapefiles = [os.path.join(train_data_path, f)
              for f in files if f.endswith('.shp')]

labeled_pixels = vectors_to_raster(shapefiles, rows, cols, geo_transform,
                                   proj)
is_train = np.nonzero(labeled_pixels)
training_labels = labeled_pixels[is_train]
training_samples = training_arr[is_train]

The class are urban and non_urban.



In [8]:
trained_classifier = RandomForestClassifier(n_jobs=-1)
trained_classifier.fit(training_samples, training_labels)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [9]:
model = pickle.dumps(trained_classifier)

## Generate tiles over Africa

In [10]:
africa = dl.places.shape("africa")
p = shapely.geometry.box(*africa.bbox)
tiles = dl.raster.dltiles_from_shape(
    resolution=500, 
    tilesize=4000, 
    pad=16, 
    shape=shapely.geometry.mapping(p))

In [ ]:
print(len(tiles['features']))

In [ ]:
pprint(tiles['features'][0])

## Prepare classification code

In [14]:
def classify_tiles(tiles, str_classifier, imagery_id):
    
    import json
    import descarteslabs as dl 
    from sklearn.ensemble import RandomForestClassifier
#     from descarteslabs.client.services import Catalog
    import pickle 
    
    classifier = pickle.loads(classifier)
    
    date = ['2016-01-01','2018-04-12']
    total_land_pixels = 0
    counter = 0
    
    for tile in tiles['features']:
        images = dl.metadata.search(
                                products=['landsat:LC08:PRE:TOAR', 'landsat:LC08:01:RT:TOAR','landsat:LC08:01:T2:TOAR'],
                                start_time=date[0],
                                end_time=date[1],
                                geom=json.dumps(tile['geometry']),
                                cloud_fraction=0.2,
                                limit = 10
                                )

        print('Tile #' + str(counter) + '. Number of scenes for this tile: ' + str(len(images['features'])))
        counter += 1
            
        ids = [f['id'] for f in images['features']]
        if ids: 
        
            arr, meta = dl.raster.ndarray(
                ids,
                bands=['red', 'green', 'blue', 'alpha'],
                scales=[[0,6000], [0, 6000], [0, 6000], None],
                data_type='Byte',
                srs = tile['properties']['cs_code'],
                resolution = tile['properties']['resolution'],
                bounds = tile['properties']['outputBounds'])

            arr = arr[16:-16, 16:-16]
            rows, cols, bands = arr.shape
            
        
            n_samples = rows*cols
            classifier = json.dumps(str_classifier)
            flat_pixels = arr.reshape((n_samples, n_bands))
            result = classifier.predict(flat_pixels)
            classification = result.reshape((rows, cols))
            print(np.unique(classification, return_counts=True))
            
            # Upload classified tile to the same product 
#             catalog = Catalog()
#            (old) catalog.upload_ndarray(classification, imagery_id, "tile_0", meta['coordinateSystem']['wkt'], geotrans=meta['geoTransform'])    
#             catalog.upload_ndarray(classification, 
#                                    product_id= imagery_id, 
#                                    image_key="tile_{}".format(counter), 
#                                    wkt_srs=meta['coordinateSystem']['wkt'], 
#                                    geotrans=meta['geoTransform'])

In [ ]:
from descarteslabs.client.services import Catalog
prod = Catalog().add_product('Urban_Africa', 
                      title='Urban_Africa', 
                      description='Urban areas identified using the random forest classificaiton.'
                     )
prod_id = prod['data']['id']

In [ ]:
Catalog().add_band(product_id= prod_id, name='urban_class', srcband=1, nbits=64,dtype='Float64',type='class',data_range=[1.000,2.000],colormap_name='magma')

## Serealize classification script and send to the DL platform

In [12]:
from descarteslabs.client.services.tasks import AsyncTasks, as_completed


at = AsyncTasks()
async_func = at.create_function(
    classify_tiles,
    name='Urban classificaiton over Africa',
    image="us.gcr.io/dl-ci-cd/images/tasks/public/geospatial/geospatial-public:latest"
)


In [13]:
task = async_func(tiles, model, '7294028cc01114d89a473cf055d29dc5cd5ffe88:Urban_Africa')